# Estimating Workplace Location

This notebook illustrates how to re-estimate ActivitySim's auto ownership model.  The steps in the process are:
  - Run ActivitySim in estimation mode to read household travel survey files, run the ActivitySim submodels to write estimation data bundles (EDB) that contains the model utility specifications, coefficients, chooser data, and alternatives data for each submodel.
  - Read and transform the relevant EDB into the format required by the model estimation package [larch](https://larch.newman.me) and then re-estimate the model coefficients.  No changes to the model specification will be made.
  - Update the ActivitySim model coefficients and re-run the model in simulation mode.
  
The basic estimation workflow is shown below and explained in the next steps.

![estimation workflow](https://github.com/RSGInc/activitysim/raw/develop/docs/images/estimation_example.jpg)

# Load libraries

In [1]:
import larch  # !conda install larch #for estimation
import pandas as pd
import numpy as np
import yaml 
import larch.util.excel
import os

# Required Inputs

In addition to a working ActivitySim model setup, estimation mode requires an ActivitySim format household travel survey.  An ActivitySim format household travel survey is very similar to ActivitySim's simulation model tables:

 - households
 - persons
 - tours
 - joint_tour_participants
 - trips (not yet implemented)

Examples of the ActivitySim format household travel survey are included in the [example_estimation data folders](https://github.com/RSGInc/activitysim/tree/develop/activitysim/examples/example_estimation).  The user is responsible for formatting their household travel survey into the appropriate format.  

After creating an ActivitySim format household travel survey, the `scripts/infer.py` script is run to append additional calculated fields.  An example of an additional calculated field is the `household:joint_tour_frequency`, which is calculated based on the `tours` and `joint_tour_participants` tables.  

The input survey files are below.

### Survey households

In [2]:
pd.read_csv("../data_sf/survey_data/override_households.csv")

,household_id,home_zone_id,income,hhsize,HHT,auto_ownership,num_workers,joint_tour_frequency
0,841891,126,48000,1,4,1,1,0_tours
1,990869,134,48000,2,1,2,2,0_tours
2,125886,113,25900,1,4,1,1,0_tours
3,727893,8,26100,2,1,0,1,0_tours
4,2741769,150,121600,4,1,2,1,0_tours
...,...,...,...,...,...,...,...,...
1995,663493,110,19180,1,6,1,1,0_tours
1996,569375,20,7400,1,6,1,0,0_tours
1997,1445193,17,75000,1,4,0,1,0_tours
1998,2833455,69,0,1,0,0,0,0_tours


### Survey persons

In [3]:
pd.read_csv("../data_sf/survey_data/override_persons.csv")

,person_id,household_id,age,PNUM,sex,pemploy,pstudent,ptype,school_zone_id,workplace_zone_id,free_parking_at_work,cdap_activity,mandatory_tour_frequency,_escort,_shopping,_othmaint,_othdiscr,_eatout,_social,non_mandatory_tour_frequency
0,166,166,54,1,2,3,3,4,-1,-1,False,N,NaN,0,0,0,0,1,0,4
1,197,197,46,1,2,3,3,4,-1,-1,False,N,NaN,0,1,0,0,0,0,16
2,268,268,46,1,1,3,3,4,-1,-1,False,N,NaN,0,0,1,1,0,0,9
3,375,375,54,1,2,3,3,4,-1,-1,False,N,NaN,0,0,1,0,0,0,8
4,387,387,44,1,2,3,3,4,-1,-1,False,N,NaN,1,0,0,1,0,0,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4400,7554799,2863464,93,1,2,3,3,5,-1,-1,False,N,NaN,0,0,0,1,0,0,1
4401,7554818,2863483,68,1,1,3,3,5,-1,-1,False,N,NaN,0,0,1,1,0,0,9
4402,7555141,2863806,93,1,2,3,3,5,-1,-1,False,N,NaN,0,2,0,1,0,0,17
4403,7555853,2864518,71,1,1,3,3,5,-1,-1,False,N,NaN,0,0,0,0,0,1,2


### Survey tours

In [4]:
pd.read_csv("../data_sf/survey_data/override_tours.csv")

,tour_id,survey_tour_id,person_id,household_id,tour_type,tour_category,destination,origin,start,end,tour_mode,survey_parent_tour_id,parent_tour_id,composition,tdd,atwork_subtour_frequency,stop_frequency
0,25820,258200,629,629,school,mandatory,12.0,131.0,12.0,21.0,WALK_HVY,NaN,NaN,NaN,121,NaN,1out_0in
1,52265,522650,1274,1274,school,mandatory,10.0,166.0,10.0,11.0,WALK_LRF,NaN,NaN,NaN,86,NaN,0out_0in
2,1117937,11179370,27266,27266,school,mandatory,12.0,9.0,17.0,18.0,WALK_LRF,NaN,NaN,NaN,163,NaN,0out_0in
3,1148523,11485230,28012,28012,school,mandatory,5.0,10.0,8.0,10.0,WALK_LRF,NaN,NaN,NaN,56,NaN,1out_0in
4,1208547,12085470,29476,29476,school,mandatory,13.0,16.0,8.0,15.0,WALK_LOC,NaN,NaN,NaN,61,NaN,1out_0in
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5773,302923726,3029237260,7388383,2749929,maint,atwork,71.0,36.0,15.0,15.0,DRIVEALONEFREE,3.029237e+09,302923742.0,NaN,145,NaN,0out_0in
5774,302942567,3029425670,7388843,2750003,eat,atwork,87.0,42.0,10.0,10.0,DRIVEALONEFREE,3.029426e+09,302942602.0,NaN,85,NaN,0out_0in
5775,302942627,3029426270,7388844,2750003,maint,atwork,67.0,17.0,14.0,14.0,WALK_LOC,3.029426e+09,302942643.0,NaN,135,NaN,0out_0in
5776,305120465,3051204650,7441962,2758909,maint,atwork,130.0,127.0,12.0,12.0,WALK,3.051205e+09,305120481.0,NaN,112,NaN,0out_1in


### Survey joint tour participants

In [5]:
pd.read_csv("../data_sf/survey_data/survey_joint_tour_participants.csv")

,participant_id,tour_id,household_id,person_id,participant_num
0,14429508701,144295087,1606646,3519392,1
1,14429508702,144295087,1606646,3519393,2
2,28367651801,283676518,2628704,6918939,1
3,28367651804,283676518,2628704,6918942,2
4,10079851903,100798519,1173905,2458502,1
...,...,...,...,...,...
223,16297928102,162979281,1769918,3975105,2
224,16297928103,162979281,1769918,3975106,3
225,16297928104,162979281,1769918,3975107,4
226,26353054902,263530549,2519358,6427575,1


# Example Setup if Needed

To avoid duplication of inputs, especially model settings and expressions, the `example_estimation` depends on the `example`.  The following commands create an example setup for use.  The location of these example setups (i.e. the folders) are important because the paths are referenced in this notebook.  The commands below download the skims.omx for the SF county example from the [activitysim resources repository](https://github.com/RSGInc/activitysim_resources).

In [6]:
!activitysim create -e example_estimation_sf -d test

# Run the Estimation Example

The next step is to run the model with an `estimation.yaml` settings file with the following settings in order to output the EDB for all submodels:

```
enable=True

bundles:
  - school_location
  - workplace_location
  - auto_ownership
  - free_parking
  - cdap
  - mandatory_tour_frequency
  - mandatory_tour_scheduling
  - joint_tour_frequency
  - joint_tour_composition
  - joint_tour_participation
  - joint_tour_destination
  - joint_tour_scheduling
  - non_mandatory_tour_frequency
  - non_mandatory_tour_destination
  - non_mandatory_tour_scheduling
  - tour_mode_choice
  - atwork_subtour_frequency
  - atwork_subtour_destination
  - atwork_subtour_scheduling
  - atwork_subtour_mode_choice
survey_tables:
  households:
    file_name: survey_data/override_households.csv
    index_col: household_id
  persons:
    file_name:  survey_data/override_persons.csv
    index_col: person_id
  tours:
    file_name:  survey_data/override_tours.csv
  joint_tour_participants:
    file_name:  survey_data/override_joint_tour_participants.csv
```

This enables the estimation mode functionality, identifies which models to run and their output estimation data bundles (EDBs), and the input survey tables, which include the override settings for each model choice.  

With this setup, the model will output an EBD with the following tables for this submodel:
  - model settings - workplace_location_model_settings.yaml
  - coefficients - workplace_location_coefficients.csv
  - utilities specification - workplace_location_SPEC.csv
  - land use data - workplace_location_landuse.csv
  - size terms - workplace_location_size_terms.csv
  - alternatives values - workplace_location_alternatives_combined.csv
  - chooser data - workplace_location_choosers_combined.csv
  
The following code runs the software in estimation mode, inheriting the settings from the simulation setup and using the San Francisco county data setup.  It produces the EDB for all submodels but runs all the model steps identified in the inherited settings file.  

In [7]:
%cd test

~/activitysim/activitysim/examples/example_estimation/notebooks/test


In [8]:
!activitysim run -c configs_estimation/configs -c configs -o output -d data_sf

# Load data and prep model for estimation

In [9]:
from activitysim.estimation.larch.location_choice import location_choice_model
model, data = location_choice_model(model_selector="school", return_data=True)

# Review data loaded from EDB

Next we can review what was read the EDB, including the coefficients, model settings, utilities specification, and chooser and alternative data.

## coefficients

In [10]:
data.coefficients

,value,constrain
coefficient_name,,
coef_univ_dist_0_1,-3.2451,F
coef_univ_dist_1_2,-2.7011,F
coef_univ_dist_2_5,-0.5707,F
coef_univ_dist_5_15,-0.5002,F
coef_univ_dist_15_up,-0.0730,F
coef_high_dist_0_1,-0.9523,F
coef_high_grade_dist_1_2,-0.5700,F
coef_high_grade_dist_2_5,-0.5700,F
coef_high_dist_5_15,-0.1930,F


## alt_values

In [11]:
data.alt_values

,person_id,variable,1,2,3,4,5,6,7,8,...,181,182,183,184,185,186,187,188,189,190
0,629,mode_choice_logsum,-1.5183555655982788,-1.353962644303794,-1.0526039754400893,-1.3907332428502703,-1.2112310544656295,-0.9024649772476414,-0.8544294523037963,-0.9528358416884833,...,-1.5077968784535976,-1.193792288653567,-1.07496629805523,-1.0762186838325454,-0.8796290051944193,-0.9219559724996267,-0.9920608655458416,-0.9485572132565794,-1.2540314052850339,-1.440460138400616
1,629,pick_count,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,629,prob,0.0,0.0,0.0,0.0,3.964160745753979e-05,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0004973126878062969,0.0,0.0,0.0502720348896301,0.0,0.0
3,629,shadow_price_size_term_adjustment,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,629,shadow_price_utility_adjustment,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16723,7541072,util_mode_choice_logsum,0.1395692158655642,0.2052734402726386,-0.03296753086863768,0.3224058911955759,0.4013527953439554,0.20770178295254108,-0.7852935252301666,0.3993011404544752,...,-0.7169849730665881,-0.6707501515058356,-0.45839788455365515,-0.41334244175244933,-0.2869641119624632,-0.46081622462339866,-0.6315359146500037,-0.4537925018083494,-0.5618093464032252,-0.7662081233221009
16724,7541072,util_no_attractions,True,True,True,True,False,True,True,True,...,True,True,True,True,False,True,True,False,True,True
16725,7541072,util_sample_of_corrections_factor,60.0,60.0,60.0,60.0,7.389915777113972,60.0,60.0,60.0,...,60.0,60.0,60.0,60.0,7.2448931528989515,60.0,60.0,2.7259268023475394,60.0,60.0
16726,7541072,util_size_variable,0.0,0.0,0.0,0.0,3.7775981687727715,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.99553683911511,0.0,0.0,9.605815100951924,0.0,0.0


## chooser_data

In [12]:
data.chooser_data

,person_id,model_choice,override_choice,home_zone_id,school_segment,household_id
0,629,13,12,131,3,629
1,1274,10,10,166,3,1274
2,27266,10,12,9,3,27266
3,28012,5,5,10,3,28012
4,29368,185,185,16,3,29368
...,...,...,...,...,...,...
979,7514214,185,185,56,3,2822879
980,7514777,185,185,106,3,2823442
981,7515185,188,188,142,3,2823850
982,7527597,69,13,188,3,2836262


## landuse

In [13]:
data.landuse

,DISTRICT,SD,county_id,TOTHH,TOTPOP,TOTACRE,RESACRE,CIACRE,TOTEMP,AGE0519,...,OPRKCST,area_type,HSENROLL,COLLFTE,COLLPTE,TOPOLOGY,TERMINAL,household_density,employment_density,density_index
zone_id,,,,,,,,,,,,,,,,,,,,,
1,1,1,1,46,82,20.3,1.0,15.00000,27318,7,...,932.83514,0,0.00000,0.00000,0.00000,3,5.89564,2.875000,1707.375000,2.870167
2,1,1,1,134,240,31.1,1.0,24.79297,42078,19,...,885.61682,0,0.00000,0.00000,0.00000,1,5.84871,5.195214,1631.374751,5.178722
3,1,1,1,267,476,14.7,1.0,2.31799,2445,38,...,716.27252,0,0.00000,0.00000,0.00000,1,5.53231,80.470405,736.891913,72.547987
4,1,1,1,151,253,19.3,1.0,18.00000,22434,20,...,314.00000,0,0.00000,0.00000,0.00000,2,5.64330,7.947368,1180.736842,7.894233
5,1,1,1,611,1069,52.7,1.0,15.00000,15662,86,...,314.01431,0,0.00000,72.14684,0.00000,1,5.52555,38.187500,978.875000,36.753679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,4,4,1,2779,8062,376.0,172.0,15.00000,1760,1178,...,0.00000,3,0.00000,0.00000,0.00000,1,2.04173,14.860963,9.411765,5.762347
187,4,4,1,1492,4139,214.0,116.0,10.00000,808,603,...,0.00000,3,0.00000,0.00000,0.00000,2,1.73676,11.841270,6.412698,4.159890
188,4,4,1,753,4072,232.0,11.0,178.00000,4502,1117,...,0.00000,2,3961.04761,17397.79102,11152.93652,1,2.28992,3.984127,23.820106,3.413233


## spec

In [14]:
data.spec

,Label,Description,Expression,university,highschool,gradeschool
0,local_dist,NaN,_DIST@skims['DIST'],1,1,1
1,util_dist_0_1,"Distance, piecewise linear from 0 to 1 miles","@_DIST.clip(0,1)",coef_univ_dist_0_1,coef_high_dist_0_1,coef_grade_dist_0_1
2,util_dist_1_2,"Distance, piecewise linear from 1 to 2 miles","@(_DIST-1).clip(0,1)",coef_univ_dist_1_2,coef_high_grade_dist_1_2,coef_high_grade_dist_1_2
3,util_dist_2_5,"Distance, piecewise linear from 2 to 5 miles","@(_DIST-2).clip(0,3)",coef_univ_dist_2_5,coef_high_grade_dist_2_5,coef_high_grade_dist_2_5
4,util_dist_5_15,"Distance, piecewise linear from 5 to 15 miles","@(_DIST-5).clip(0,10)",coef_univ_dist_5_15,coef_high_dist_5_15,coef_grade_dist_5_15
5,util_dist_15_up,"Distance, piecewise linear for 15+ miles",@(_DIST-15.0).clip(0),coef_univ_dist_15_up,coef_high_dist_15_up,coef_grade_dist_15_up
6,util_no_attractions,No attractions,@df['size_term']==0,-999,-999,-999
7,util_mode_choice_logsum,Mode choice logsum,mode_choice_logsum,coef_mode_logsum,coef_mode_logsum,coef_mode_logsum
8,util_sample_of_corrections_factor,Sample of alternatives correction factor,"@np.minimum(np.log(df.pick_count/df.prob), 60)",1,1,1


## size_spec

In [15]:
data.size_spec

,AGE0519,HSENROLL,COLLFTE,COLLPTE
segment,,,,
university,0.0,0.0,0.592,0.408
gradeschool,1.0,0.0,0.000,0.000
highschool,0.0,1.0,0.000,0.000


# Estimate

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  Larch has two built-in estimation methods: BHHH and SLSQP.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.  SLSQP is safer, but slower, and may need additional iterations.

In [16]:
model.estimate(method='BHHH', options={'maxiter':1000})

req_data does not request avail_ca or avail_co but it is set and being provided


,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
-999,-999.000000,-999.0,-999.0,-999.0,-999.0,1,,-999.000000
1,1.000000,1.0,1.0,1.0,1.0,1,,1.000000
coef_grade_dist_0_1,-1.824840,0.0,0.0,NaN,NaN,0,,-1.824840
coef_grade_dist_15_up,-0.046000,0.0,0.0,NaN,NaN,0,,-0.046000
coef_grade_dist_5_15,-0.144663,0.0,0.0,NaN,NaN,0,,-0.144663
coef_high_dist_0_1,-2.106211,0.0,0.0,NaN,NaN,0,,-2.106211
coef_high_dist_15_up,-0.188200,0.0,0.0,NaN,NaN,0,,-0.188200
coef_high_dist_5_15,-0.101747,0.0,0.0,NaN,NaN,0,,-0.101747
coef_high_grade_dist_1_2,-0.751852,0.0,0.0,NaN,NaN,0,,-0.751852
coef_high_grade_dist_2_5,-0.667119,0.0,0.0,NaN,NaN,0,,-0.667119


~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 4.654468147935182e-17 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 4.295335610251568e-16 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 8.610772823633788e-16 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 1.5691224886738212e-16 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 3.9490381039538154e-16 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 7.341243915088908e-16 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
~/larch/larch/

┣          loglike: -4058.719620083745
┣                x: -999                       -999.000000
┃                   1                             1.000000
┃                   coef_grade_dist_0_1          -1.824840
┃                   coef_grade_dist_15_up        -0.046000
┃                   coef_grade_dist_5_15         -0.144663
┃                   coef_high_dist_0_1           -2.106211
┃                   coef_high_dist_15_up         -0.188200
┃                   coef_high_dist_5_15          -0.101747
┃                   coef_high_grade_dist_1_2     -0.751852
┃                   coef_high_grade_dist_2_5     -0.667119
┃                   coef_mode_logsum              0.372245
┃                   coef_univ_dist_0_1           -3.311744
┃                   coef_univ_dist_15_up         -0.073000
┃                   coef_univ_dist_1_2           -2.878224
┃                   coef_univ_dist_2_5           -0.456009
┃                   coef_univ_dist_5_15          -0.572553
┃                   gradeschool_AGE0519           0.000000
┃                   highschool_HSENROLL           0.000000
┃                   university_COLLFTE           -0.524249
┃                   university_COLLPTE           -1.208828
┃                   dtype: float64
┣        tolerance: 7.272237307434155e-06
┣            steps: array([1., 1., 1., 1., 1., 1.])
┣          message: 'Optimization terminated successfully.'
┣     elapsed_time: datetime.timedelta(microseconds=108888)
┣           method: 'BHHH'
┣          n_cases: 984
┣ iteration_number: 6
┣          logloss: 4.124715061060717

### Estimated coefficients

In [17]:
model.parameter_summary()

,Value,Std Err,t Stat,Signif,Null Value,Constrained
-999,-999.,NA,NA,,-999.00,fixed value
1,1.00,NA,NA,,1.00,fixed value
coef_grade_dist_0_1,-1.82,0.435,-4.19,***,0.00,
coef_grade_dist_15_up,-0.0460,9.98e-16,-46112362455645.20,***,0.00,
coef_grade_dist_5_15,-0.145,0.0638,-2.27,*,0.00,
coef_high_dist_0_1,-2.11,0.801,-2.63,**,0.00,
coef_high_dist_15_up,-0.188,5.09e-15,-36939300202814.84,***,0.00,
coef_high_dist_5_15,-0.102,0.143,-0.71,,0.00,
coef_high_grade_dist_1_2,-0.752,0.186,-4.05,***,0.00,
coef_high_grade_dist_2_5,-0.667,0.0510,-13.09,***,0.00,


# Output Estimation Results

In [18]:
from activitysim.estimation.larch import update_coefficients, update_size_spec
result_dir = data.edb_directory/"estimated"

## Write updated utility coefficients

In [19]:
update_coefficients(
    model, data, result_dir,
    output_file="school_location_coefficients_revised.csv",
);

## Write updated size coefficients

In [20]:
model.pf

,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best,std_err,t_stat,robust_std_err,robust_t_stat,unconstrained_std_err,unconstrained_t_stat,constrained
-999,-999.000000,-999.0,-999.0,-999.0,-999.0,1,,-999.000000,NaN,NaN,0.000000e+00,NaN,0.000000e+00,NaN,fixed value
1,1.000000,1.0,1.0,1.0,1.0,1,,1.000000,NaN,NaN,0.000000e+00,NaN,0.000000e+00,NaN,fixed value
coef_grade_dist_0_1,-1.824840,0.0,0.0,NaN,NaN,0,,-1.824840,4.350583e-01,-4.194471e+00,4.377099e-01,-4.169062e+00,4.350583e-01,-4.194471e+00,
coef_grade_dist_15_up,-0.046000,0.0,0.0,NaN,NaN,0,,-0.046000,9.975633e-16,-4.611236e+13,9.874153e-16,-4.658628e+13,9.975633e-16,-4.611236e+13,
coef_grade_dist_5_15,-0.144663,0.0,0.0,NaN,NaN,0,,-0.144663,6.375309e-02,-2.269115e+00,5.954350e-02,-2.429537e+00,6.375309e-02,-2.269115e+00,
coef_high_dist_0_1,-2.106211,0.0,0.0,NaN,NaN,0,,-2.106211,8.009670e-01,-2.629585e+00,7.819521e-01,-2.693530e+00,8.009670e-01,-2.629585e+00,
coef_high_dist_15_up,-0.188200,0.0,0.0,NaN,NaN,0,,-0.188200,5.094845e-15,-3.693930e+13,4.982247e-15,-3.777412e+13,5.094845e-15,-3.693930e+13,
coef_high_dist_5_15,-0.101747,0.0,0.0,NaN,NaN,0,,-0.101747,1.432817e-01,-7.101153e-01,1.466306e-01,-6.938969e-01,1.432817e-01,-7.101153e-01,
coef_high_grade_dist_1_2,-0.751852,0.0,0.0,NaN,NaN,0,,-0.751852,1.855920e-01,-4.051102e+00,1.904776e-01,-3.947195e+00,1.855920e-01,-4.051102e+00,
coef_high_grade_dist_2_5,-0.667119,0.0,0.0,NaN,NaN,0,,-0.667119,5.098153e-02,-1.308550e+01,5.079096e-02,-1.313460e+01,5.098153e-02,-1.308550e+01,


In [21]:
update_size_spec(
    model, data, result_dir, 
    output_file="school_location_size_terms.csv",
)

,segment,model_selector,TOTHH,RETEMPN,FPSEMPN,HEREMPN,OTHEMPN,AGREMPN,MWTEMPN,AGE0519,HSENROLL,COLLFTE,COLLPTE
0,work_low,workplace,0.000000,0.129129,0.193193,0.383383,0.120120,0.010010,0.164164,0.000,0.000000,0.000000,0.000000
1,work_med,workplace,0.000000,0.120120,0.197197,0.325325,0.139139,0.008008,0.210210,0.000,0.000000,0.000000,0.000000
2,work_high,workplace,0.000000,0.110000,0.207000,0.284000,0.154000,0.006000,0.239000,0.000,0.000000,0.000000,0.000000
3,work_veryhigh,workplace,0.000000,0.093000,0.270000,0.241000,0.146000,0.004000,0.246000,0.000,0.000000,0.000000,0.000000
4,university,school,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.664760,0.335240
5,gradeschool,school,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000,0.000000,0.000000,0.000000
6,highschool,school,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,1.000000,0.000000,0.000000
7,escort,non_mandatory,0.000000,0.225000,0.000000,0.144000,0.000000,0.000000,0.000000,0.465,0.166000,0.000000,0.000000
8,shopping,non_mandatory,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
9,eatout,non_mandatory,0.000000,0.742000,0.000000,0.258000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000


## Write the model estimation report, including coefficient t-statistic and log likelihood

In [22]:
model.to_xlsx(
    result_dir/"school_location_model_estimation.xlsx", 
    data_statistics=False,
);

# Next Steps

The final step is to either manually or automatically copy the `workplace_location_coefficients_revised.csv` file and `workplace_location_size_terms.csv` file to the configs folder, rename them to `workplace_location_coefficients.csv` and `destination_choice_size_terms.csv`, and run ActivitySim in simulation mode.

In [23]:
pd.read_csv(result_dir/"school_location_coefficients_revised.csv")

,coefficient_name,value,constrain
0,coef_univ_dist_0_1,-3.311744,F
1,coef_univ_dist_1_2,-2.878224,F
2,coef_univ_dist_2_5,-0.456009,F
3,coef_univ_dist_5_15,-0.572553,F
4,coef_univ_dist_15_up,-0.073000,F
5,coef_high_dist_0_1,-2.106211,F
6,coef_high_grade_dist_1_2,-0.751852,F
7,coef_high_grade_dist_2_5,-0.667119,F
8,coef_high_dist_5_15,-0.101747,F
9,coef_high_dist_15_up,-0.188200,F


In [24]:
pd.read_csv(result_dir/"school_location_size_terms.csv")

,index,segment,model_selector,TOTHH,RETEMPN,FPSEMPN,HEREMPN,OTHEMPN,AGREMPN,MWTEMPN,AGE0519,HSENROLL,COLLFTE,COLLPTE
0,0,work_low,workplace,0.000000,0.129129,0.193193,0.383383,0.120120,0.010010,0.164164,0.000,0.000000,0.000000,0.000000
1,1,work_med,workplace,0.000000,0.120120,0.197197,0.325325,0.139139,0.008008,0.210210,0.000,0.000000,0.000000,0.000000
2,2,work_high,workplace,0.000000,0.110000,0.207000,0.284000,0.154000,0.006000,0.239000,0.000,0.000000,0.000000,0.000000
3,3,work_veryhigh,workplace,0.000000,0.093000,0.270000,0.241000,0.146000,0.004000,0.246000,0.000,0.000000,0.000000,0.000000
4,4,university,school,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.664760,0.335240
5,5,gradeschool,school,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000,0.000000,0.000000,0.000000
6,6,highschool,school,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,1.000000,0.000000,0.000000
7,7,escort,non_mandatory,0.000000,0.225000,0.000000,0.144000,0.000000,0.000000,0.000000,0.465,0.166000,0.000000,0.000000
8,8,shopping,non_mandatory,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
9,9,eatout,non_mandatory,0.000000,0.742000,0.000000,0.258000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
